In [1]:
# version of the algorithm used for testing

import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import graphviz as viz
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

dfA=pd.read_csv('Arxiv_Normalized.csv')
print('done')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


done


In [2]:
titles = dfA['abstract'].to_list()
vect = TfidfVectorizer(min_df=1, stop_words="english")
tfidf_init = vect.fit_transform(titles)
print('done')
    

done


In [8]:
def top_three(df, keyword1, keyword2=None, keyword3=None, topic1=None, topic2=None, figures=None, pages=None, target = None):

    temp = df.copy()
    
    temp['score'], sim1, sim2, sim3 = [0 for j in temp['title']], [0 for j in temp['title']], [0 for j in temp['title']], [0 for j in temp['title']]

    tfidf1 = vect.transform([keyword1])

    sim1 = cosine_similarity(tfidf_init, tfidf1).flatten()

    sim1 = np.round(sim1, decimals=3)

    if keyword2 != None:
       tfidf2 = vect.transform([keyword1])
       sim2 = cosine_similarity(tfidf_init, tfidf2).flatten()
       sim2 = np.round(sim2, decimals=3) 

    if keyword3 != None:
       tfidf3 = vect.transform([keyword3])
       sim3 = cosine_similarity(tfidf_init, tfidf3).flatten()
       sim3 = np.round(sim3, decimals=3) 
    
    temp['score'] = temp['score'] + sim1 + sim2 + sim3

    temp = temp.sort_values(by='score', ascending = False)

    temp.reset_index(drop=True, inplace=True)

    temp = temp.iloc[0:5000]

    temp['update_date'] = pd.to_datetime(df.update_date)

    for i in range(0, 5000):
                    
        if topic1 != None:
            if topic1 == temp['Category 1'][i] or topic1 == temp['Category 2'][i] or topic1 == temp['Category 3'][i]:
                temp['score'][i] = temp['score'][i] * 1.5
        if topic2 != None:
            if topic2 == temp['Category 1'][i] or topic2 == temp['Category 2'][i] or topic2 == temp['Category 3'][i]:
                temp['score'][i] = temp['score'][i] * 1.5

        if figures != None:
            if temp['Figures'][i] >= figures:
                temp['score'][i] = temp['score'][i] * 1.5

        if pages != None:
            if temp['Pages'][i] <= pages:
                temp['score'][i] = temp['score'][i] + 0.2**np.absolute(pages - int(temp['Pages'][i]))

        if target != None:
                temp['score'][i] = temp['score'][i] + 0.2**np.absolute(int(target) - temp['update_date'][i].year)

    sorted = temp.sort_values(by='score', ascending = False)

    sorted.reset_index(drop=True, inplace=True)

    sorted = sorted.iloc[0:1000]
 
    return sorted

df2 = top_three(dfA, 'artificial intelligence and chatgpt')

print(df2['score'])

authors = {}

for k in range(1, 6):
    for i, j in enumerate(df2['Author ' + str(k)]):
        if pd.isna(j) == False:
            if j in authors:
                authors[j] += df2['score'][i]
            else:
                authors[j] = df2['score'][i]

aut = dict(sorted(authors.items(), key=lambda item: item[1], reverse=True))

print(aut)

common = list(aut.items())[:3]

merged_dfs = []
fit_lists = []

nodeCounts = []
edgeCounts = []
all_vals = []

for i in range(0,len(common)):
    df2.fillna(value='0', inplace=True)

    a = df2[df2['Author 1'].str.contains(common[i][0])]
    b = df2[df2['Author 2'].str.contains(common[i][0])]
    c = df2[df2['Author 3'].str.contains(common[i][0])]
    d = df2[df2['Author 4'].str.contains(common[i][0])]
    e = df2[df2['Author 5'].str.contains(common[i][0])]

    merged = pd.merge(a, b, how='outer')
    merged = pd.merge(merged, b, how='outer')
    merged = pd.merge(merged, c, how='outer')
    merged = pd.merge(merged, d, how='outer')
    merged = pd.merge(merged, e, how='outer')

    print(merged)

    titles = merged['abstract']
    vect2 = TfidfVectorizer(min_df=1, stop_words="english")
    tfidf = vect2.fit_transform(titles)
    pairwise_similarity = tfidf * tfidf.T

    array = pairwise_similarity.toarray()
    np.fill_diagonal(array, 0)
    array = np.round(array, decimals=3) 

    fits = []
    fit_vals = []

    for i in range(0, len(array)):
        sfits = []
        if(len(array)>1):
            if (array[i][np.nanargmax(array[i])] not in fit_vals):
                sfits.append(np.nanargmax(array[i]))
                fit_vals.append(array[i][sfits[0]])
            for m, f in enumerate(array[i]):
                if f > 0.25 and m != np.nanargmax(array[i]):
                    if(f not in fit_vals):
                        sfits.append(m)
                        fit_vals.append(f)
        else:
            sfits.append(0)
        print(sfits)
        fits.append(sfits)
    merged_dfs.append(merged)
    fit_lists.append(fits)

    all_vals.append(fit_vals)

bgraph = viz.Graph('round-table', format='svg', graph_attr={'center':'True', 'colorscheme':'blues9', 'K':'0.8', 'label': "test", 'labelloc':'t'}) 
bgraph.attr(layout='fdp')
bgraph.attr(fontsize='50')
bgraph.attr(size='100,100')

for i in range(0,len(common)):
    graph = viz.Graph('cluster_'+str(i), format='svg', graph_attr={'center':'True', 'colorscheme':'blues9', 'K':'0.3', 'label': common[i][0], 'labelloc':'t'}) 
    graph.attr(layout='fdp')
    graph.attr(fontsize='50')
    graph.attr(size='100,100')

    NC = 0
    EC = 0

    for j in range(0, len(merged_dfs[i])):
        graph.node(str(j), merged_dfs[i]['title'][j], URL='https://arxiv.org/abs/' + str(merged_dfs[i]['id'][j]), style = 'filled', color = 'black', fillcolor ='steelblue2', fontsize = '15')
        NC += 1
        for n in range(0, len(fit_lists[i][j])):
            graph.edge(str(j), str(fit_lists[i][j][n]), constraint='false')
            EC += 1
    nodeCounts.append(NC)
    edgeCounts.append(EC)
    
    graph.render(directory='doctest-output', view=True)    

average = [np.mean(all_vals[0]), np.mean(all_vals[1]), np.mean(all_vals[2])]

hold = pd.DataFrame({'NC':nodeCounts, 'EC':edgeCounts, 'Fits':average})

prior = pd.read_csv('Test Results.csv')

prior = pd.concat([prior, hold])

prior = prior[['NC', 'EC', 'Fits']]

prior.to_csv('Test Results.csv')

print('done')






0      0.732
1      0.682
2      0.657
3      0.642
4      0.621
       ...  
995    0.137
996    0.137
997    0.137
998    0.137
999    0.137
Name: score, Length: 1000, dtype: float64
{'Marcus Hutter': 3.481, 'Tshilidzi Marwala': 3.2329999999999997, 'Feng Liu': 2.517, 'Yong Shi': 2.517, 'Shane Legg': 2.151, 'Laveen Kanal': 2.067, 'John Lemmer': 2.067, 'Dimiter Dobrev': 1.927, 'Uwe Aickelin': 1.855, 'Iztok Fister': 1.6520000000000001, 'Jose Hernandez-Orallo': 1.5859999999999999, 'Javier Insa-Cabrera': 1.5350000000000001, 'Sven Koenig': 1.3439999999999999, 'J Gerard Wolff': 1.328, 'Ying Liu': 1.137, 'Viktoras Veitas': 1.112, 'David Weinbaum': 1.1119999999999999, 'Mark O. Riedl': 1.084, 'Hans J. Briegel': 1.01, 'Eric Horvitz': 0.96, 'David Heckerman': 0.9149999999999999, 'Evan Hurwitz': 0.895, 'Vedran Dunjko': 0.869, 'A. M. Zagoskin': 0.865, 'J. Gerard Wolff': 0.859, 'Utku Kose': 0.813, 'Yu. A. Pashkin': 0.81, 'O. Astafiev': 0.8099999999999999, 'Joel Veness': 0.791, 'Adi Makmal': 0.782, 

/var/folders/9d/7b80y3ns221bt1hc8j0s0rsm0000gn/T/ipykernel_3808/940421196.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'bob' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.fillna(value='bob', inplace=True)


done


In [55]:
import pandas as pd

hold = pd.DataFrame({'NC':[0], 'EC':[0], 'Fits':[0]})

hold.to_csv('Test Results.csv')

In [ ]:
from difflib import SequenceMatcher as SM

str1 = 'But I have promises to keep, and miles to go before I sleep.'
str2 = 'But I have many promises to keep, and miles to run before sleep.'

similarity_ratio = SM(None, str1, str2,).ratio() # 0.9032258064516129
similarity_ratio

collect = []

for i in range (0,100000):
    if(SM(None, df['title'][i], 'SINDY').ratio() > 0.2):
        collect.append(i)

print(collect)

df.loc[28968]

In [6]:
test = df[df['Author 3'] == 'Kyle J. LaFollette']

test

,Unnamed: 0,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,...,Author 1,Author 2,Author 3,Author 4,Author 5,Category 1,Category 2,Category 3,Pages,Figures


In [5]:
df=df[df['id','title', 'journal-ref', 'doi', 'report-no', 'Author 1', 'Author 2', 'Author 3', 'Author 4', 'Author 5', 'Category 1', 'Category 2', 'Category 3', 'Pages', 'Figures']]
df

KeyError: ('id', 'title', 'journal-ref', 'doi', 'report-no', 'Author 1', 'Author 2', 'Author 3', 'Author 4', 'Author 5', 'Category 1', 'Category 2', 'Category 3', 'Pages', 'Figures')